In [1]:
%load_ext autoreload

In [2]:
%autoreload
from dotenv import load_dotenv
from src.data import scrape_genius_lyrics as scr
import os
import json, re
from tqdm import tqdm

In [3]:
load_dotenv()
GENIUS = os.getenv("GENIUS")

## Sample pull of single song lyric

In [46]:
hit = scr.return_top_hit(['kaskade', 'angel on my shoulders'], token=GENIUS)
hit_url = scr.extract_url(hit)
hit_lyrics = scr.extract_lyrics(hit_url)

In [47]:
hit

{'highlights': [],
 'index': 'song',
 'result': {'annotation_count': 1,
  'api_path': '/songs/1685152',
  'full_title': 'Angel On My Shoulder by\xa0Kaskade (Ft.\xa0Tamra\xa0Keenan)',
  'header_image_thumbnail_url': 'https://images.genius.com/891ddd4aa82671b7f8346e4489c82c02.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/891ddd4aa82671b7f8346e4489c82c02.700x700x1.jpg',
  'id': 1685152,
  'lyrics_owner_id': 1549345,
  'lyrics_state': 'complete',
  'path': '/Kaskade-angel-on-my-shoulder-lyrics',
  'primary_artist': {'api_path': '/artists/20775',
   'header_image_url': 'https://images.genius.com/2a860b9ec22082e43e5a7592c7ef7f13.650x366x1.jpg',
   'id': 20775,
   'image_url': 'https://images.genius.com/dc9ecdd86df7a81037c362187ecab67f.1000x1000x1.jpg',
   'is_meme_verified': False,
   'is_verified': False,
   'name': 'Kaskade',
   'url': 'https://genius.com/artists/Kaskade'},
  'pyongs_count': None,
  'song_art_image_thumbnail_url': 'https://images.genius.com/891ddd4aa8267

In [5]:
print(hit_lyrics)



[Verse 1]
It's time you see
The love you give
Is killing me
And I want to live
This isn't right
Search your soul for light

[Pre-Chorus]
So I'm turning left
I told myself
I could be as bad as someone else
But I find it hard
When angel's on my right

[Chorus]
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand

[Verse 2]
Why won't you go
No means no
Your dirty life
Is like a one man show
You're a night attack
You turn the light to black

[Pre-Chorus]
So I'm turning left
I told myself
I could be as bad as someone else
But I find it hard
When angel's on my right

[Chorus]
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand

[Bridge]
Talk to me
Walk with

## Use MSD to build search terms

In [6]:
import pandas as pd
ut = '../data/external/MillionSongSubset/AdditionalFiles/subset_unique_tracks.txt'
df = pd.read_csv(ut, sep='<SEP>', header=None, engine='python',
                 names=['trackid', 'songid', 'artist', 'title'])
df.head()

,trackid,songid,artist,title
0,TRAAAAW128F429D538,SOMZWCG12A8C13C480,Casual,I Didn't Mean To
1,TRAAABD128F429CF47,SOCIWDW12A8C13D406,The Box Tops,Soul Deep
2,TRAAADZ128F9348C2E,SOXVLOJ12AB0189215,Sonora Santanera,Amor De Cabaret
3,TRAAAEF128F4273421,SONHOTT12A8C13493C,Adam Ant,Something Girls
4,TRAAAFD128F92F423A,SOFSOCN12A8C143F5D,Gob,Face the Ashes


Check the number of artist and titles that have parentheses.

In [7]:
xx = list(df.title.astype(str))
yy = list(df.artist.astype(str))

In [8]:
r = re.compile('\([^)]+\)')
pars = list(filter(r.search, xx))
print(len(pars))
pars = list(filter(r.search, yy))
print(len(pars))

1820
11


Extract MSD track id and search terms based on arist and title.

Remove parentheses

In [9]:
import re

def remove_parentheses(phrase):
    return re.sub(r'\([^)]+\)', '', phrase).strip()

songs = []
for row in df.itertuples(index=False):
    artist = remove_parentheses(str(row.artist).lower())
    title = remove_parentheses(str(row.title).lower())
    tup = (row.trackid, [artist, title])
    songs.append(tup)
# songs

Use Genius search API to retrieve top hit for search term and store response with all meta data

In [10]:
len(songs)

10000

In [11]:
songs[:50]

[('TRAAAAW128F429D538', ['casual', "i didn't mean to"]),
 ('TRAAABD128F429CF47', ['the box tops', 'soul deep']),
 ('TRAAADZ128F9348C2E', ['sonora santanera', 'amor de cabaret']),
 ('TRAAAEF128F4273421', ['adam ant', 'something girls']),
 ('TRAAAFD128F92F423A', ['gob', 'face the ashes']),
 ('TRAAAMO128F1481E7F', ['jeff and sheri easter', 'the moon and i']),
 ('TRAAAMQ128F1460CD3', ['rated r', 'keepin it real']),
 ('TRAAAPK128E0786D96', ['tweeterfriendly music', 'drop of rain']),
 ('TRAAARJ128F9320760', ['planet p project', 'pink world']),
 ('TRAAAVG12903CFA543', ['clp', 'insatiable']),
 ('TRAAAVO128F93133D4', ['jennyanykind', 'young boy blues']),
 ('TRAABCL128F4286650', ['wayne watson', 'the urgency']),
 ('TRAABDL12903CAABBA', ['andy andy', 'la culpa']),
 ('TRAABJL12903CDCF1A', ['bob azzam', 'auguri cha cha']),
 ('TRAABJV128F1460C49', ['lionel richie', 'tonight will be alright']),
 ('TRAABLR128F423B7E3', ['blue rodeo', 'floating']),
 ('TRAABNV128F425CEE1', ['richard souther', 'high tide

In [12]:
scr.get_metadata(songs[:50], token=GENIUS, outfile='../data/interim/test_outfile.csv', batchsize=15)

100%|██████████| 50/50 [00:34<00:00,  1.45it/s]


True

In [11]:
scr.get_lyrics('../data/interim/genius_metadata.csv', outfile='../data/interim/test_lyrics.csv')

100%|██████████| 20/20 [00:20<00:00,  1.05s/it]


In [8]:
# import pandas as pd
# import numpy as np
# import csv

# def get_lyrics(meta_file, outfile):

#     # get line count for progress bar
#     with open(meta_file, 'r') as f:
#         row_count = sum(1 for row in f)
#     row_count -= 1
    
#     # stream metadata file and output line by line
#     with open(meta_file, 'r') as f:
#         rd = csv.reader(f)
#         # skip header
#         next(rd)
#         for row in tqdm(rd, total=row_count):
#             if row[2]: # if URL exists
#                 msd_id = row[0]
#                 url = row[2]
#                 track_lyrics = scr.extract_lyrics(url)
#                 tup = (msd_id, track_lyrics)

#                 scr.create_lyrics_file(outfile)
#                 with open(outfile, 'a', newline='') as f:
#                     wt = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
#                     wt.writerow(tup)
                    
# get_lyrics('../data/interim/genius_metadata.csv', outfile='../data/interim/test_lyrics.csv')

100%|██████████| 20/20 [00:10<00:00,  1.93it/s]


In [6]:
# with open('../data/interim/genius_metadata.csv', 'r') as f:
#     rd = csv.reader(f)
#     next(rd)
#     for row in tqdm(rd, total=20):
#         if row[2]:
#             msd_id = row[0]
#             url = row[2]
#             track_lyrics = scr.extract_lyrics(url)
#             tup = (msd_id, track_lyrics)

#             scr.create_lyrics_file('../data/interim/test_lyrics.csv')
#             with open('../data/interim/test_lyrics.csv', 'a', newline='') as f:
#                 wt = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
#                 wt.writerow(tup)

100%|██████████| 20/20 [00:01<00:00, 10.66it/s]


In [52]:
import pandas as pd
import numpy as np
import csv

def get_lyrics(meta_file, outfile, batchsize=100):
    
    
    # get line count for progress bar
    with open(meta_file, 'r') as f:
        row_count = sum(1 for row in f)
        
    row_count -= 1
        
    df = pd.read_csv(meta_file, chunksize=batchsize)

    for chunk in tqdm(df, total=4):
        songs = []
        for row in chunk.itertuples(index=False):
            msd_id = row.msd_id
            song_url = row.song_url
            tup = (msd_id, song_url)
            songs.append(tup)

        lyrics = []
        for song in songs:
            track_id = song[0]
            if song[1] is not np.nan:
                track_lyrics = scr.extract_lyrics(song[1])
                tup = (track_id, track_lyrics)
            else:
                tup = (track_id, None)
            lyrics.append(tup)

        scr.create_lyrics_file(outfile)
        with open(outfile, 'a', newline='') as f:
            wt = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
            wt.writerows(lyrics)
            
get_lyrics('../data/interim/genius_metadata.csv', outfile='../data/interim/test_lyrics.csv', batchsize=5)


100%|██████████| 4/4 [00:09<00:00,  2.69s/it]


In [10]:
with open('../data/interim/genius_metadata.json', 'r') as f:
    genius = json.load(f)

In [15]:
genius[5]

['TRAAAMO128F1481E7F', None]

In [21]:
if genius[5][1] is not None:
    scr.extract_url(genius[5][1])

In [24]:
with open('../data/interim/genius_lyrics.json', 'r') as f:
    lyr = json.load(f)

In [25]:
lyr[:5]

[['TRAAAAW128F429D538',
  "\n\n[Verse 1]\nAlright, I might\nHave had a little glare when I stared at ya ho\nBut I didn't know she was like that\nShe stared right back\nMy niggas warnin me that she was comin on to me\nI react like a mack do, I act cool\nJust to test her, 'cause I'm no jester\nI suggest her and her friend be outtie\nBecause I don't want to make my pal get rowdy\nAnd doubt me, our friendship\nBut when lips touch, I go crazy in the clutch\nSorta like a schitzo I forgets my bros and pals over gals\nI didn't mean to, but when you fiend you do\nStrange things for the boot denim no matter who's in em\nGrab a flooze then I'm in traffic\nDon't laugh it might be your girl that I'm talkin about\nI didn't mean to\n\n[Chorus 4X]\nIt ain't my fault that your girl got caught\nIt ain't my fault that your girl got caught (I didn't mean to)\n\n[Verse 2]\nAnother incident when I went way beyond\nWhat I shoulda done\nJohn shoulda stopped before them drawers dropped\nBut I didn't, ain't no 

In [38]:
lyrics_length = []
for n, i in enumerate(lyr):
    if i[1] is not None:
        tup = (n, i[0],len(i[1]))
    else:
        tup = (n, i[0], None)
    lyrics_length.append(tup)

In [39]:
lyrics_length

[(0, 'TRAAAAW128F429D538', 2217),
 (1, 'TRAAABD128F429CF47', 1136),
 (2, 'TRAAADZ128F9348C2E', 477),
 (3, 'TRAAAEF128F4273421', 1113),
 (4, 'TRAAAFD128F92F423A', 604),
 (5, 'TRAAAMO128F1481E7F', None),
 (6, 'TRAAAMQ128F1460CD3', 18370),
 (7, 'TRAAAPK128E0786D96', None),
 (8, 'TRAAARJ128F9320760', 886),
 (9, 'TRAAAVG12903CFA543', 13318),
 (10, 'TRAAAVO128F93133D4', None),
 (11, 'TRAABCL128F4286650', None),
 (12, 'TRAABDL12903CAABBA', 1203),
 (13, 'TRAABJL12903CDCF1A', 346648),
 (14, 'TRAABJV128F1460C49', 978),
 (15, 'TRAABLR128F423B7E3', 1122),
 (16, 'TRAABNV128F425CEE1', 10280),
 (17, 'TRAABRB128F9306DD5', 64716),
 (18, 'TRAABVM128F92CA9DC', 1546),
 (19, 'TRAABXG128F9318EBD', 8889),
 (20, 'TRAABYN12903CFD305', None),
 (21, 'TRAABYW128F4244559', None),
 (22, 'TRAACCG128F92E8A55', 98165),
 (23, 'TRAACER128F4290F96', 1403),
 (24, 'TRAACFV128F935E50B', 1304),
 (25, 'TRAACHN128F1489601', 6181),
 (26, 'TRAACIW12903CC0F6D', None),
 (27, 'TRAACLV128F427E123', 75364),
 (28, 'TRAACNS128F14A2DF5'

In [71]:
lyr[7]

['TRAAAPK128E0786D96', None]

In [72]:
genius[7]

['TRAAAPK128E0786D96', None]

In [84]:
songs[42]

('TRAADLN128F14832E9', ['the rolling stones', 'angie'])

In [75]:
hit = scr.return_top_hit(['roberto torres', 'sabor guajiro'], token=GENIUS)
hit_url = scr.extract_url(hit)
hit_lyrics = scr.extract_lyrics(hit_url)

AttributeError: 'NoneType' object has no attribute 'get'

In [80]:
hit = scr.return_top_hit(['the rolling stones', 'angie'], token=GENIUS)
hit

{'highlights': [],
 'index': 'song',
 'result': {'annotation_count': 7,
  'api_path': '/songs/140056',
  'full_title': 'Angie by\xa0The\xa0Rolling Stones',
  'header_image_thumbnail_url': 'https://images.rapgenius.com/bd05b68f416c12912766c8ff6fa6f5fc.222x227x1.jpg',
  'header_image_url': 'https://images.rapgenius.com/bd05b68f416c12912766c8ff6fa6f5fc.222x227x1.jpg',
  'id': 140056,
  'lyrics_owner_id': 27699,
  'lyrics_state': 'complete',
  'path': '/The-rolling-stones-angie-lyrics',
  'primary_artist': {'api_path': '/artists/774',
   'header_image_url': 'https://images.genius.com/1176553983a0b98b9ea8a7c4d5b90cdf.850x315x1.jpg',
   'id': 774,
   'image_url': 'https://images.genius.com/23bbf05f7ee8286a89057205b480a766.620x387x1.jpg',
   'is_meme_verified': False,
   'is_verified': False,
   'name': 'The Rolling Stones',
   'url': 'https://genius.com/artists/The-rolling-stones'},
  'pyongs_count': 9,
  'song_art_image_thumbnail_url': 'https://images.rapgenius.com/bd05b68f416c12912766c8ff6